# Analysis: Illusory Pitch

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# Set file paths
TRIAL_DATAFILE = '../data/response_data.csv'
SCORES_DATAFILE = '../data/scores.csv'
FIGURE_PATH = './figures/'

# Set colors
color1 = '#512D6D'
color2 = '#F8485E'

# Set line widths
lw = 1.5
capsize = 3
msize = 7

# Set fonts
small = 12
medium = 16
large = 20
plt.rc('font', size=small)          # controls default text sizes
plt.rc('axes', titlesize=large)     # fontsize of the axes title
plt.rc('axes', labelsize=medium)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=small)    # fontsize of the tick labels
plt.rc('ytick', labelsize=small)    # fontsize of the tick labels
plt.rc('legend', fontsize=small)    # legend fontsize
plt.rc('figure', titlesize=large)  # fontsize of the figure title
matplotlib.rcParams['axes.spines.right'] = False
matplotlib.rcParams['axes.spines.top'] = False

### Load and preprocess data

In [ ]:
# Load scores
scores = pd.read_csv(SCORES_DATAFILE)

# Loftus & Masson (1994) method of plotting within-subject effects
subj_means = scores.groupby(['subject']).mean().reset_index()
grand_mean = subj_means.mean()
within_scores = scores.copy()
for i, subj in enumerate(subj_means.subject):
    within_scores.loc[within_scores.subject == subj, 'dprime'] -= subj_means.dprime[i] - grand_mean.dprime
    within_scores.loc[within_scores.subject == subj, 'C'] -= subj_means.C[i] - grand_mean.C

In [ ]:
# Stagger the x-axis values so error bars will be easier to read
within_scores.loc[within_scores.octave == 3, 'offset'] -= .5
within_scores.loc[within_scores.octave == 5, 'offset'] += .5

### Plot bias and sensitivity

In [ ]:
###
# Bias
###

plt.subplot(121)

# Plot average C per condition
plt.axhline(0, ls='--', c='k', alpha=.5)
sns.lineplot(x=within_scores.offset[within_scores.octave == 3]-.5, y='C', data=within_scores[within_scores.octave == 3],
             ls='-', lw=lw, marker='o', ms=msize, color=color1, label='A3 (220 Hz)',
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))
sns.lineplot(x=within_scores.offset[within_scores.octave == 5]+.5, y='C', data=within_scores[within_scores.octave == 5],
             ls='-', lw=lw, marker='o', ms=msize, color=color2, label='A5 (880 Hz)',
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))

# Stylize subplot
plt.xlim(-20, 20)
plt.xticks([-15, 0, 15], ['15% Early', 'On Time', '15% Late'])
plt.ylim(-.6, 1)
plt.xlabel('Probe Timing Offset')
plt.ylabel('Bias ($C$) Towards "Low"')


###
# Sensitivity
###

plt.subplot(122)

# Plot average d' per condition
plt.axhline(0, ls='--', c='k', alpha=.5)
sns.lineplot(x=within_scores.offset[within_scores.octave == 3]+.5, y='dprime', data=within_scores[within_scores.octave == 3],
             ls='-', lw=lw, marker='o', ms=msize, color=color1, label='A3 (220 Hz)',
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))
sns.lineplot(x=within_scores.offset[within_scores.octave == 5]-.5, y='dprime', data=within_scores[within_scores.octave == 5],
             ls='-', lw=lw, marker='o', ms=msize, color=color2, label='A5 (880 Hz)',
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))

# Sylize subplot
plt.xlim(-20, 20)
plt.xticks([-15, 0, 15], ['15% Early', 'On Time', '15% Late'])
plt.ylim(-.5, 3)
plt.xlabel('Probe Timing Offset')
plt.ylabel('Sensitivity ($d\prime$)')

# Stylize figure and save
plt.gcf().set_size_inches(9, 3.5)
plt.tight_layout()
plt.gcf().savefig(FIGURE_PATH + 'scores.svg')

In [ ]:
# Load trial data
data = pd.read_csv(TRIAL_DATAFILE)

In [ ]:
# Exclude trials with reaction times slower than 10 seconds
bad_rt = data.rt > 10000
print('Excluding %s percent of trials for outlier RT' % (bad_rt.mean() * 100))
data = data[~bad_rt]

In [ ]:
# Loftus & Masson (1994) method of plotting within-subject effects
# For reaction time plots, we will analyze at the trial level rather than subject level due to some participants 
# having very few incorrect responses, giving us poor estimates of their mean reaction times for incorrect trials
subj_means = data.groupby('subject').mean().reset_index()
grand_mean = subj_means.mean()
within_data = data.copy()
for i, subj in enumerate(subj_means.subject):
    within_data.loc[within_data.subject == subj, 'rt'] -= subj_means.rt[i] - grand_mean.rt

In [ ]:
# Stagger the x-axis values so error bars will be easier to read
within_data.loc[within_data.pitch_shift == '+', 'offset'] -= .5
within_data.loc[within_data.pitch_shift == '-', 'offset'] += .5

In [ ]:
plt.subplot(121)

sns.lineplot(x='offset', y='rt', hue='pitch_shift', data=within_data[within_data['correct']],
             ls='-', lw=lw, marker='o', ms=msize, hue_order=['+', '-'], palette=[color1, color2],
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))

plt.title('Correct Response')
plt.xlabel('Probe Timing Offset')
plt.ylabel('Reaction Time (ms)')
plt.legend(['Up', 'Down'], title='Pitch Shift', loc='upper left')
plt.xlim(-20, 20)
plt.xticks([-15, 0, 15], ['15% Early', 'On Time', '15% Late'])
plt.ylim(800, 1200)

plt.subplot(122)
sns.lineplot(x='offset', y='rt', hue='pitch_shift', data=within_data[~within_data['correct']],
             ls='-', lw=lw, marker='o', ms=msize, hue_order=['+', '-'], palette=[color1, color2],
             err_style='bars', err_kws=dict(capsize=capsize, lw=lw, capthick=lw))

# Stylize subplot
plt.title('Incorrect Response')
plt.xlabel('Probe Timing Offset')
plt.ylabel('Reaction Time (ms)')
plt.legend(['Up', 'Down'], title='Pitch Shift', loc='upper right')
plt.xlim(-20, 20)
plt.xticks([-15, 0, 15], ['15% Early', 'On Time', '15% Late'])
plt.ylim(1000, 1800)

# Stylize figure and save
plt.gcf().set_size_inches(9, 4.5)
plt.tight_layout()
plt.gcf().savefig(FIGURE_PATH + 'rt.svg')